In [6]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(readxl)
library(dplyr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

ERROR: Error in library(themis): there is no package called ‘themis’


In [2]:
data <- read_csv("https://raw.githubusercontent.com/thisisyifulin/group_project-group34-/main/Training_Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.csv") |>
select(STG, SCG, STR, LPR, PEG, UNS)

new_data <- data |>
select(STG, STR, LPR, PEG) # selecting the variables

A_grade <- filter(new_data, 1.00 > PEG, PEG > 0.80) |> # changing PEG from decimals to letter grades
mutate(PEG = "A")

B_grade <- filter(new_data, 0.79 > PEG, PEG > 0.65) |> 
mutate(PEG = "B")

C_grade <- filter(new_data, 0.64 > PEG, PEG > 0.55) |> 
mutate(PEG = "C")

D_grade <- filter(new_data, 0.54 > PEG, PEG > 0.50) |> 
mutate(PEG = "D")

F_grade <- filter(new_data, 0.49 > PEG, PEG > 0.00) |> 
mutate(PEG = "F")

mutated_data <- bind_rows(A_grade, B_grade, C_grade, D_grade, F_grade)

mutated_data

New names:
• `` -> `...7`
• `` -> `...8`
Rows: 258 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): UNS, Attribute Information:
dbl (5): STG, SCG, STR, LPR, PEG
lgl (2): ...7, ...8

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


STG,STR,LPR,PEG
<dbl>,<dbl>,<dbl>,<chr>
0.08,0.10,0.24,A
0.00,0.50,0.20,A
0.18,0.55,0.30,A
⋮,⋮,⋮,⋮
0.72,0.45,0.79,F
0.54,0.55,0.64,F
0.50,0.81,0.61,F


In [3]:
set.seed(2345)
#split data into training set & testing set
mutated_data_split <- initial_split(mutated_data, prop = 0.75, strata = PEG)
data_training <- training(mutated_data_split)
data_testing <- testing(mutated_data_split)

data_training


STG,STR,LPR,PEG
<dbl>,<dbl>,<dbl>,<chr>
0.08,0.10,0.24,A
0.00,0.50,0.20,A
0.18,0.55,0.30,A
⋮,⋮,⋮,⋮
0.58,0.31,0.29,F
0.72,0.45,0.79,F
0.54,0.55,0.64,F


In [4]:
#count the amount of each class of PEG to check if the classes are balanced
counts_class <- data_training |>
                group_by(PEG) |>
                summarize(count_class = n())
counts_class

PEG,count_class
<chr>,<int>
A,21
B,28
C,23
D,6
F,94


In [8]:
install.packages("themis")
install.packages("RANN")
install.packages("ROSE")
library(themis)

peg_recipe <- recipe(PEG ~ ., data = data_training) |>
  step_upsample(PEG, over_ratio = 1, skip = FALSE) |>
  prep()

peg_recipe

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



Recipe

Inputs:

      role #variables
   outcome          1
 predictor          3

Training data contained 172 data points and no missing data.

Operations:

Up-sampling based on PEG [trained]

In [10]:
upsampled_data_training <- bake(peg_recipe, data_training)

counts_class_again <- upsampled_data_training |>
                group_by(PEG) |>
                summarize(count_class = n())

counts_class_again

PEG,count_class
<fct>,<int>
A,94
B,94
C,94
D,94
F,94
